In [1]:
import numpy as np
import altair as alt 
import pandas as pd

In [2]:
def init_parameters(n_features, n_neurons, n_output): 
    np.random.seed(100)
    W1 = np.random.uniform(size = (n_features, n_neurons))
    b1 = np.random.uniform(size = (1, n_neurons))
    W2 = np.random.uniform(size = (n_neurons, n_output))
    b2 = np.random.uniform(size = (1, n_output))

    return {"W1": W1, "b1": b1, "W2": W2, "b2": b2}


In [3]:
def linear_function(W, X, b): 
    return (X @ W) + b 

In [4]:
def sigmoid_func(Z): 
    return 1 / (1 + np.exp(-Z))

In [5]:
def cost_function(A, y):
    epsilon = 1e-15  
    A = np.clip(A, epsilon, 1 - epsilon)
    return -np.mean(y * np.log(A) + (1 - y) * np.log(1 - A))


In [6]:
def predict(X, W1, W2, b1, b2): 
    Z1 = linear_function(W1, X, b1)
    S1 = sigmoid_func(Z1)
    Z2 = linear_function(W2, S1, b2)
    S2 = sigmoid_func(Z2)
    return np.where(S2 >= 0.5, 1, 0)


In [7]:
def fit(X, y, n_features=2, n_neurons=3, n_output=1, iterations=5000, eta=0.1): 
    params = init_parameters(n_features, n_neurons, n_output)
    errors = [] 

    for _ in range(iterations): 
        # Forward pass
        Z1 = linear_function(params['W1'], X, params['b1'])
        S1 = sigmoid_func(Z1)
        Z2 = linear_function(params['W2'], S1, params['b2'])
        S2 = sigmoid_func(Z2)

        # Costo
        error = cost_function(S2, y)
        errors.append(error)

        # Backpropagation con cross-entropy
        delta2 = S2 - y
        W2_gradients = S1.T @ delta2 
        params["W2"] -= W2_gradients * eta
        params["b2"] -= np.sum(delta2, axis=0, keepdims=True) * eta 

        delta1 = (delta2 @ params["W2"].T) * S1 * (1 - S1)
        W1_gradients = X.T @ delta1 
        params["W1"] -= W1_gradients * eta 
        params["b1"] -= np.sum(delta1, axis=0, keepdims=True) * eta 

    return errors, params 


In [8]:
y = np.array([[0, 1, 1, 0]]).T 
X = np.array([[0, 0, 1, 1],
              [0, 1, 0, 1]]).T 

In [9]:
errors, params = fit(X, y)

In [10]:
y_pred = predict(X, params["W1"], params["W2"], params["b1"], params["b2"])
accuracy = (y_pred == y).mean() * 100
print(f"Multi-layer perceptron accuracy: {accuracy:.2f}%")

Multi-layer perceptron accuracy: 100.00%


In [11]:
alt.data_transformers.disable_max_rows()
df = pd.DataFrame({"errors": errors, "time-step": np.arange(len(errors))})
alt.Chart(df).mark_line().encode(x="time-step", y="errors").properties(title='Loss NumPy')


alt.Chart(...)

# Mismo Dataset en Framework Pytorch

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import altair as alt

In [14]:
# Inputs: XOR problem
X = np.array([[0, 0, 1, 1],
              [0, 1, 0, 1]]).T
y = np.array([[0, 1, 1, 0]]).T

In [15]:
# Convert to torch tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32)

In [16]:
# Define the model
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.sigmoid(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

model = SimpleNet()

In [17]:
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

In [18]:
# Training loop
errors = []
epochs = 3000

for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    loss.backward()
    optimizer.step()
    errors.append(loss.item())

In [19]:
# Convert errors to DataFrame and plot
df2 = pd.DataFrame({"errors": errors, "time-step": np.arange(epochs)})
alt.Chart(df2).mark_line().encode(
    x="time-step",
    y="errors"
).properties(title='Loss PyTorch')


alt.Chart(...)

1. ¿Existen cambios de arquitectura en las 2 redes implementadas?
    - No, en ambas arquitecturas se tiene una capa oculta y usan la activación sigmoide. La red de PyTorch tiene 16 neuronas ocultas y 3 en Numpy, pero la estructura general es la misma.
2. ¿Exiten diferencias en la velocidad de convergencia entre las 2 redes?
    - Si, Pytorch converge más rápido debido al optimizador adam y al manejo numérico. Por otro lado, NumPy usa descenso de gradiente manual con tasa de aprendizaje fija.
3. En la función init_parameters, ¿qué sucede si inicializamos los pesos en 0 en lugar de valores aleatorios? ¿y los bias?
    - Si los pesos se inicializan en 0, todos los nodos aprenderán lo mismo, esto causa problemas de simetría y el modelo no aprenderá
    - Si solo los bias se inicializan en 0, el modelo aún puede aprender correctamete. 
    - Es común inicializar biases en 0 pero no los pesos.
